<a href="https://colab.research.google.com/github/profteachkids/chetools/blob/main/tools/IsopropanolSynthesis_che5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -N -q https://raw.githubusercontent.com/profteachkids/chetools/main/tools/che4.ipynb -O che5.ipynb
!pip install importnb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 1.4 MB/s eta 0:00:00


In [ ]:
from importnb import Notebook
with Notebook():
    from che5 import Range, Comp, CompArray, RangeArray, DotDict, d2nt, Props

In [ ]:
import jax
import jax.numpy as jnp
from scipy.optimize import root

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [ ]:
# Static parameters (Total feed, feed mole fradtions, feed temperature and )
# 1. propene
# 2. diisopropyl ether
# 3. isopropanol
# 4. water
# Rx1:  C3H6 + H2O -> C3H8O
# Rx2:  2C3H8O -> C6H14 + H2O

#Simple version - Overall mass balance only

e = DotDict()
e.alkeneF = 100.
e.waterF = 100.

e.alkeney = jnp.array([1., 0., 0., 0.])
e.waterx = jnp.array([0., 0., 0., 1.])

e.rx1 = jnp.array([-1, 0, 1, -1])
e.rx2 = jnp.array([0, 1, -2, 1])

e.rx1_extent = Range(e.alkeneF/2, 0., e.alkeneF)
e.rx2_extent = Range(e.alkeneF/4, 0., e.alkeneF/2)

e.alcoholP = Range(e.alkeneF/2, 0., e.alkeneF)
e.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

e.etherP = Range(e.alkeneF/2, 0., e.alkeneF)
e.etherPx = jnp.array([0.0005, 0.98, 0.018, 0.0015])

In [ ]:
x0, x2nt, wrap, x2unk, const  = d2nt(e)

In [ ]:
def eqs(d):
    r=DotDict()
    feed = d.alkeneF*d.alkeney + d.waterF*d.waterx
    product = (d.etherP*d.etherPx + d.alcoholP*d.alcoholPx)

    molecular_balance = feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - product

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return molecular_balance, r

In [ ]:
eqs_wrapped = wrap(eqs)
eqs_wrapped(x0)

Array([ 49.975, -24.25 , -47.9  ,  72.175], dtype=float64)

In [ ]:
sol=root(eqs_wrapped, x0, jac=jax.jit(jax.jacobian(eqs_wrapped)))
print(sol)

 message: The solution converged.
 success: True
  status: 1
     fun: [ 2.460e-12  8.819e-11 -1.425e-10  2.289e-11]
       x: [ 1.063e+01 -2.145e+00  2.975e+00 -2.974e+00]
    nfev: 30
    njev: 2
    fjac: [[-8.077e-02 -7.905e-02  9.898e-01 -8.637e-02]
           [-1.438e-01  7.005e-01  1.045e-01  6.912e-01]
           [-8.836e-01  2.265e-01 -8.892e-02 -4.000e-01]
           [-4.382e-01 -6.722e-01 -3.746e-02  5.957e-01]]
       r: [ 3.810e-02 -9.426e+00 -3.417e+00  2.247e-01  5.193e+00
           -5.103e-01 -2.563e+00  4.171e-01 -8.483e-01  2.468e+00]
     qtf: [-1.561e-08  1.059e-09  1.175e-09 -2.605e-10]


In [ ]:
nt=x2nt(sol.x)
nt._asdict()

{'alkeneF': Array(100., dtype=float64),
 'waterF': Array(100., dtype=float64),
 'alkeney': Array([1., 0., 0., 0.], dtype=float64),
 'waterx': Array([0., 0., 0., 1.], dtype=float64),
 'rx1': Array([-1,  0,  1, -1], dtype=int64),
 'rx2': Array([ 0,  1, -2,  1], dtype=int64),
 'rx1_extent': Array(99.9975704, dtype=float64),
 'rx2_extent': Array(5.23773658, dtype=float64),
 'alcoholP': Array(95.14322505, dtype=float64),
 'alcoholPx': Array([0.   , 0.005, 0.94 , 0.055], dtype=float64),
 'etherP': Array(4.85920455, dtype=float64),
 'etherPx': Array([5.0e-04, 9.8e-01, 1.8e-02, 1.5e-03], dtype=float64)}

In [ ]:
unk=x2unk(sol.x)
unk._asdict()

{'rx1_extent': Array(99.9975704, dtype=float64),
 'rx2_extent': Array(5.23773658, dtype=float64),
 'alcoholP': Array(95.14322505, dtype=float64),
 'etherP': Array(4.85920455, dtype=float64)}

In [ ]:
const._asdict()

{'alkeneF': Array(100., dtype=float64),
 'waterF': Array(100., dtype=float64),
 'alkeney': Array([1., 0., 0., 0.], dtype=float64),
 'waterx': Array([0., 0., 0., 1.], dtype=float64),
 'rx1': Array([-1,  0,  1, -1], dtype=int64),
 'rx2': Array([ 0,  1, -2,  1], dtype=int64),
 'alcoholPx': Array([0.   , 0.005, 0.94 , 0.055], dtype=float64),
 'etherPx': Array([5.0e-04, 9.8e-01, 1.8e-02, 1.5e-03], dtype=float64)}

In [ ]:
_, r= eqs(x2nt(sol.x))
r

{'alkene': Array([100.,   0.,   0.,   0.], dtype=float64),
 'water': Array([  0.,   0.,   0., 100.], dtype=float64),
 'ether': Array([2.42960227e-03, 4.76202046e+00, 8.74656819e-02, 7.28880682e-03],      dtype=float64),
 'alcohol': Array([ 0.        ,  0.47571613, 89.43463155,  5.23287738], dtype=float64)}

<img src='https://github.com/profteachkids/CHE2064_Spring2022/raw/main/IsopropylSynthesisProcessDiagram.jpg'>

In [ ]:
# Include internal flows (recycle)


d=DotDict()
d.alkeneF = 100.
d.waterF = 100.

d.alkeney = jnp.array([1., 0., 0., 0.])
d.waterx = jnp.array([0., 0., 0., 1.])

d.rx1 = jnp.array([-1, 0, 1, -1])
d.rx2 = jnp.array([0, 1, -2, 1])

d.rx1_extent = Range(d.alkeneF/2, 0., d.alkeneF)
d.rx2_extent = Range(d.alkeneF/4, 0., d.alkeneF/2)

d.alcoholP = Range(d.alkeneF/2, 0., d.alkeneF)
d.alcoholPx = jnp.array([0., 0.005, 0.94, 0.055])

d.etherP = Range(d.alkeneF/2, 0., d.alkeneF)
d.etherPx = jnp.array([0.0005, 0.98, 0.018, 0.0015])

d.WARFR = 12. #water_alkene_reactor_feed_ratio

d.D1F = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.D1Fx = Comp([0.001, 0.03, 0.07, 0.9])

d.D1K = jnp.array([100., 10., 5., 0.2])
d.D1Vy = Comp([0.01, 0.299, 0.3, 0.3])
d.D1Lx = Comp([0.0001, 0.1, 0.1, 0.8])
d.D1V = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)
d.D1L = Range(d.alkeneF * d.WARFR/2, 0., 2.*d.WARFR*d.alkeneF)

d.decantW = Range(d.alkeneF * d.WARFR, 0., 2.*d.WARFR*d.alkeneF)
d.decantWx = jnp.array([0.0001, 0.0005, 0.0014, 0.98])


In [ ]:
x0, x2nt, wrap, x2unk, const  = d2nt(d)

In [ ]:
def eqs2(d):
    r=DotDict()
    r.reactor_feed = d.alkeneF*d.alkeney + d.waterF*d.waterx + d.D1L*d.D1Lx + d.decantW*d.decantWx

    r.REACTOR = r.reactor_feed + d.rx1_extent*d.rx1 + d.rx2_extent*d.rx2 - d.D1F*d.D1Fx

    r.FEED_RATIO = jnp.atleast_1d((d.waterF+d.D1L+d.decantW)/d.alkeneF - d.WARFR)

    r.D1 = d.D1F*d.D1Fx - d.D1V*d.D1Vy - d.D1L*d.D1Lx
    r.D1K = d.D1Vy/d.D1Lx - d.D1K

    r.AZ = d.D1V*d.D1Vy - d.etherP*d.etherPx - d.alcoholP*d.alcoholPx - d.decantW*d.decantWx

    r.alkene = d.alkeneF*d.alkeney
    r.water = d.waterF*d.waterx
    r.ether = d.etherP*d.etherPx
    r.alcohol = d.alcoholP*d.alcoholPx
    return jnp.concatenate([r.REACTOR, r.FEED_RATIO, r.D1, r.D1K, r.AZ]), r


In [ ]:
eqs2_wrapped = wrap(eqs2)

In [ ]:
sol=root(eqs2_wrapped, x0, jac=jax.jit(jax.jacobian(eqs2_wrapped)))
sol.message

'The solution converged.'

In [ ]:
unk=x2unk(sol.x)
unk._asdict()

{'rx1_extent': Array(99.9975704, dtype=float64),
 'rx2_extent': Array(5.23773658, dtype=float64),
 'alcoholP': Array(95.14322505, dtype=float64),
 'etherP': Array(4.85920455, dtype=float64),
 'D1F': Array(1199.74805577, dtype=float64),
 'D1Fx': Array([3.50840994e-06, 8.53998248e-03, 2.16963767e-01, 7.74492742e-01],      dtype=float64),
 'D1Vy': Array([3.37442255e-05, 4.60555454e-02, 7.86283218e-01, 1.67627493e-01],      dtype=float64),
 'D1Lx': Array([3.37442255e-07, 4.60555454e-03, 1.57256644e-01, 8.38137465e-01],      dtype=float64),
 'D1V': Array(113.87993521, dtype=float64),
 'D1L': Array(1085.86812057, dtype=float64),
 'decantW': Array(14.13187943, dtype=float64)}

In [ ]:
eqs2_wrapped(sol.x)

Array([ 4.56206253e-14, -3.55271368e-15, -5.68434189e-14, -1.13686838e-13,
        0.00000000e+00,  4.63930110e-16,  2.66453526e-15,  0.00000000e+00,
        2.27373675e-13, -4.26325641e-14,  1.77635684e-15,  0.00000000e+00,
        0.00000000e+00, -9.08561421e-17, -4.47558657e-16, -1.36626821e-14,
       -1.77635684e-15], dtype=float64)